In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api

In [14]:
!pip install gensim
!pip install glove-python-binary

ERROR: Could not find a version that satisfies the requirement glove-python-binary (from versions: none)
ERROR: No matching distribution found for glove-python-binary


In [5]:
df = pd.read_csv("db_mod.csv")
df.head()

,uid,tags,statement,rating
0,1895G,"dp,flows,greedy",You are given a string consisting of character...,3100
1,1895F,"combinatorics,dp",Let's call an array a of n non-negative intege...,2600
2,1895E,"dp,games,graphs,greedy,sortings",Monocarp and Bicarp are playing a card game. E...,2300
3,1895D,bitmasks,"You are given n-1 integers a_1, a_2, \dots, a_...",1900
4,1895C,dp,A ticket is a non-empty string of digits from ...,1400


In [6]:
problem_statement = df.statement.apply(gensim.utils.simple_preprocess)
problem_statement

,statement
0,"[you, are, given, string, consisting, of, char..."
1,"[let, call, an, array, of, non, negative, inte..."
2,"[monocarp, and, bicarp, are, playing, card, ga..."
3,"[you, are, given, integers, a_, a_, dots, a_, ..."
4,"[ticket, is, non, empty, string, of, digits, f..."
...,...
5499,"[permutation, is, an, ordered, group, of, numb..."
5500,"[simon, has, rectangular, table, consisting, o..."
5501,"[dima, is, good, person, in, fact, he, great, ..."
5502,"[dima, liked, the, present, he, got, from, inn..."


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [8]:
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Keep only words not in stop_words
    return " ".join(filtered_words)  # Join the filtered words back into a string
df['statement'] = df['statement'].apply(remove_stopwords)

In [9]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    words = text.split()  # Split the text into words
    lemmas = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    return " ".join(lemmas)  # Join the lemmas back into a string
df['statement'] = df['statement'].apply(lemmatize_text)

In [10]:
import re
def remove_math_symbols(text):
    # Define a pattern to match common mathematical symbols
    pattern = r"[^\w\s]"  # Matches any character that is not a word character or whitespace

    # Remove the matched symbols using re.sub()
    cleaned_text = re.sub(pattern, "", text)
    return cleaned_text
df['statement'] = df['statement'].apply(remove_math_symbols)

In [11]:
def remove_two_letter_words(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if len(word) > 2]  # Keep only words with length greater than 2
    return " ".join(filtered_words)  # Join the filtered words back into a string
df['statement'] = df['statement'].apply(remove_two_letter_words)

In [12]:
def remove_words_with_numbers(text):
    # Define a pattern to match words containing numbers
    pattern = r"\b\w*\d\w*\b"  # Matches whole words containing at least one digit

    # Remove the matched words using re.sub()
    cleaned_text = re.sub(pattern, "", text)
    return cleaned_text
df['statement'] = df['statement'].apply(remove_words_with_numbers)

In [22]:
import gensim.downloader as api
glove_vectors = api.load("glove-wiki-gigaword-300")  # Loads 300-dimensional GloVe embeddings



[==================================================] 100.0% 376.1/376.1MB downloaded


In [31]:
import numpy as np
from collections import defaultdict

# Initialize vocabulary and co-occurrence matrix
vocab = set()
co_occurrence = defaultdict(float)

window_size = 4
# Iterate through the corpus to build vocabulary and co-occurrence matrix
for sentence in df["statement"]:
    words = sentence.split()
    for i in range(len(words)):
        word = words[i]
        vocab.add(word)
        for j in range(max(0, i - window_size), min(i + window_size + 1, len(words))):
            if i != j:
                co_occurrence[(word, words[j])] += 1.0 / abs(i - j)


In [32]:
embedding_dim = 300
word_embeddings = {
    word: np.random.randn(embedding_dim) for word in vocab
}

In [34]:
learning_rate = 0.05  # Reduced learning rate
num_epochs = 7

# Gradient descent to update word embeddings
for epoch in range(num_epochs):
    total_loss = 0
    for (word_i, word_j), observed_count in co_occurrence.items():
        # Calculate dot product of word embeddings
        dot_product = np.dot(word_embeddings[word_i], word_embeddings[word_j])

        # Calculate difference and update, handling potential overflow
        diff = dot_product - np.log(observed_count + 1e-8)  # Add small value to avoid log(0)
        total_loss += 0.5 * diff**2

        # Gradient clipping to prevent exploding gradients
        gradient = diff * word_embeddings[word_j]
        gradient = np.clip(gradient, -1.0, 1.0)  # Clip gradient values between -1 and 1

        word_embeddings[word_i] -= learning_rate * gradient

    print(f"Epoch: {epoch+1}, Loss: {total_loss}")

Epoch: 1, Loss: nan
Epoch: 2, Loss: nan
Epoch: 3, Loss: nan
Epoch: 4, Loss: nan
Epoch: 5, Loss: nan
Epoch: 6, Loss: nan
Epoch: 7, Loss: nan


In [48]:
glove_vectors.most_similar("good")

[('better', 0.765958309173584),
 ('really', 0.7361074686050415),
 ('always', 0.7173537611961365),
 ('you', 0.7073798775672913),
 ('well', 0.7045711278915405),
 ('excellent', 0.703949511051178),
 ('very', 0.6966387033462524),
 ('things', 0.6936353445053101),
 ('think', 0.6899343729019165),
 ('way', 0.6838910579681396)]

In [38]:
def rev_embed(x):  # Pass the model as an argument
    x = x.split(' ')
    embedd = []
    for i in x:
        if i in glove_vectors:  # Check if word is in vocabulary
            embedd.append(glove_vectors[i]) # size  of model.wv[i] is 100
    if embedd:  # If any embeddings were found
        x = np.array(embedd)
        x = np.mean(x, axis=0)
        return x
    else:
        return np.zeros(glove_vectors.vector_size)  # Return a zero vector for missing words


In [39]:

feat_vect_glove = []
for i in df['statement']:
  feat_vect_glove.append(rev_embed(i))

In [42]:
vect_glove_hash = {}
for i in range(len(df)):
  name = str(df["rating"][i])
  vect_glove_hash[name] = rev_embed(df["statement"][i]).tolist()

vect_glove_hash

{'3100': [-0.09041310101747513,
  0.05213411897420883,
  -0.09135767072439194,
  -0.06033935770392418,
  -0.018646592274308205,
  0.10069023072719574,
  0.018214251846075058,
  -0.0020246331114321947,
  0.018442612141370773,
  -1.0663208961486816,
  0.10088992863893509,
  -0.07421929389238358,
  -0.005748743657022715,
  0.09361308813095093,
  0.05676211789250374,
  -0.001001341501250863,
  -0.13429786264896393,
  0.018859855830669403,
  0.01686761900782585,
  0.046848997473716736,
  -0.0013680927222594619,
  0.16012249886989594,
  0.18318107724189758,
  0.054049182683229446,
  -0.03377649560570717,
  0.010370103642344475,
  0.029614169150590897,
  -0.016220849007368088,
  0.02976902574300766,
  0.037087198346853256,
  0.15189892053604126,
  0.20903147757053375,
  -0.1470067948102951,
  0.03908837214112282,
  -0.6876996755599976,
  0.09758072346448898,
  -0.0949590727686882,
  -0.08883976191282272,
  -0.08531522750854492,
  -0.03915489464998245,
  -0.08017764985561371,
  -0.047125287353

In [43]:
feat_vect_glove=np.array(feat_vect_glove)
feat_vect_glove

array([[-0.02813098,  0.14080231, -0.05836291, ..., -0.09472144,
         0.06772012, -0.15265749],
       [-0.2009128 ,  0.19257449,  0.14554343, ..., -0.26006526,
        -0.01561763, -0.17041837],
       [-0.06794499,  0.44243437, -0.0630985 , ..., -0.26578882,
        -0.0250913 , -0.22581549],
       ...,
       [-0.09351466,  0.14261651, -0.13240537, ..., -0.14218335,
        -0.00975577, -0.03971072],
       [-0.1732254 ,  0.10837323, -0.08893785, ..., -0.06526554,
         0.04543452, -0.14043014],
       [-0.23505585,  0.12592585,  0.0088101 , ..., -0.26058319,
        -0.08033445, -0.06766014]])

In [44]:
feat_vect_glove.shape


(5504, 300)